# Import / 라이브러리 호출

**이번 대회에서는 train_test_split 함수를 사용하여 모델성능을 검증하는 방법을 사용했습니다.**

In [1]:
import os
import random
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import f1_score

import warnings
warnings.filterwarnings("ignore")

# 랜덤시드 고정

**매번 고정된 결과를 얻기 위해서 사용합니다.  
seed를 고정하지 않는다면 같은 코드라도 매번 다른 결과가 나오게됩니다.  
항상 동일한 결과를 얻기 위해서 사용합니다.**

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

seed_everything(42) # Seed 고정

# train.csv, test.csv 불러오기

In [3]:
train = pd.read_csv("./train.csv")
test = pd.read_csv("./test.csv")

In [4]:
# train.csv 파일의 데이터를 확인합니다.
train.head(3)

,ID,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration,genre
0,TRAIN_00000,0.3341,0.9231,10,0.8540,0.3797,0.008,0.0002,0.7697,0.7902,0.5391,175,trap
1,TRAIN_00001,0.6344,0.5955,10,0.2826,0.1378,0.000,0.7137,0.1597,0.4162,0.3043,513,techno
2,TRAIN_00002,0.3983,0.1107,8,0.2459,0.0247,0.774,0.0000,0.5105,0.4291,0.2324,151,Trap Metal


# 독립변수(X), 종속변수(y)로 나누기

**모델을 학습하기 위해서는 독립변수와 종속변수로 나누어야 합니다.  
일반적으로 모델에는 정수, 실수형태의 값만 사용할 수 있습니다.  
분류 문제일 경우 target(해당 데이터에서는 genre)에 한해 문자열 사용이 가능합니다.  
이외의 문자열 데이터인 'ID'는 제거하겠습니다.**

In [5]:
# X는 독립변수이므로 종속변수를 제거합니다. 또한 target 이외의 문자열 데이터를 제거합니다.
X = train.drop(["ID", "genre"], axis = 1)
# y는 종속변수로 값을 설정합니다.
y = train[['genre']]

# train에서와 마찬가지로 문자열이 포함된 특성은 제거합니다.
test = test.drop(["ID"], axis = 1)

**모델의 성능을 중간적으로 점검하기 위해 검증데이터(validation data)를 생성하겠습니다.**

In [6]:
# 학습데이터, 검증데이터 분리
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size = 0.2)

# 모델 정의  
**RandomForest는 tree기반 모델로, DecisionTree를 여러 개 생성하여 그들의 평균으로 예측의 성능을 높이는 Ensemble Machine Learning 모델입니다.  
RandomForest에 대해 더 자세히 알고싶으시다면 아래의 링크를 참고해주시기 바랍니다.**


**[Lv2 모델링 1/2 python 파이썬 랜덤포레스트 개념, 선언](https://dacon.io/competitions/open/235698/talkboard/403632)**

In [7]:
model = RandomForestClassifier(random_state = 42)

# 모델 학습

In [8]:
# 학습데이터를 모델에 입력합니다.
model.fit(X_train,y_train)

RandomForestClassifier(random_state=42)

# 모델 검증  
**모델이 어느정도 정답을 맞추는지 검증을 통해 대략적으로 확인할 수 있습니다.  
대회 평가 지표인 Macro-F1 Score로 검증을 수행하겠습니다.  
모델에 검증데이터를 입력하여 검증 점수를 예측합니다.**

In [9]:
val_pred = model.predict(X_valid)

# Macro f1 score을 사용하기 위해 average 인자 값을 "macro" 로 설정해줍니다.
print("현재 Macro F1 Score의 검증 점수는 {}입니다.".format(f1_score(val_pred, y_valid, average = "macro")))

현재 Macro F1 Score의 검증 점수는 0.6261751859544912입니다.


# 예측

In [10]:
# 최종 예측을 하기위해 test값을 입력합니다.
pred = model.predict(test)

# 제출

In [11]:
# 제출 파일을 불러옵니다.
submission = pd.read_csv("./sample_submission.csv")
submission["genre"] = pred

In [12]:
submission

,ID,genre
0,TEST_00000,Hiphop
1,TEST_00001,Dark Trap
2,TEST_00002,Emo
3,TEST_00003,techhouse
4,TEST_00004,Pop
...,...,...
16917,TEST_16917,techhouse
16918,TEST_16918,Hiphop
16919,TEST_16919,hardstyle
16920,TEST_16920,psytrance


In [13]:
# 해당 파일을 다운로드 받아서 제출해주세요.
submission.to_csv("./submit.csv", index = False)